In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import statistics


from sklearn import preprocessing

%matplotlib inline

In [2]:
PATH_TO_SUBSIDIES = r'data\Данные Меры поддержки на сайте ГИСП.csv'
PATH_TO_ALLOCATORS = r'data\Реестр субсидий\allocators.json'
PATH_TO_RECIEVERS = r'data\Реестр субсидий\receivers.json'

ENCODING = 'cp1251'

In [3]:
types = {'h': np.int64,
        'i': np.int64,
        'j': np.int64}
dfs = pd.read_excel(r'data\orgs-clean.xlsx', dtype=types)
dfs.head()

,ОКВЭД2,"Вид деятельности, основной ТАСС","Вид деятельности, дополнительный ТАСС",Организационно правовая форма,Тип компании,Атрибуты предприятия,Отрасль,ОГРН,ИНН,КПП,Регион
0,Производство элементов электронной аппаратуры ...,Производство элементов электронной аппаратуры ...,"Деятельность вспомогательная прочая, связанная...",НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],"Предприятие, оказывающее существенное влияние ...",Радиоэлектронная промышленность [258],1027700140930,7.735011e+09,773501001,Москва [3505]
1,Производство оружия и боеприпасов [27574],Производство оружия и боеприпасов [27575],Производство коммуникационного оборудования [2...,АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026719],Организация [7026691],Системообразующее предприятие [8690579] / Орга...,Радиоэлектронная промышленность [258],1027739001993,7.731084e+09,997450001,Москва [3505]
2,Производство оружия и боеприпасов [27575],Производство оружия и боеприпасов [27575],Лесоводство и прочая лесохозяйственная деятель...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Организация ОПК [8780665] / Системообразующее ...,Промышленность боеприпасов и спецхимии [256],1127154020311,7.105516e+09,710501001,Тульская область [3535]
3,Производство красителей и пигментов [27354] / ...,Производство оружия и боеприпасов [27575],Производство красителей и пигментов [27354] / ...,ФЕДЕРАЛЬНЫЕ ГОСУДАРСТВЕННЫЕ УНИТАРНЫЕ ПРЕДПРИЯ...,Организация [7026691],Одобрено Минпромторгом России [8690599] / Орга...,Промышленность боеприпасов и спецхимии [256],1034700561895,4.703010e+09,470350001,Ленинградская область [3500]
4,Производство коммуникационного оборудования [2...,Научные исследования и разработки в области ес...,Производство коммуникационного оборудования [2...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Системообразующее предприятие [8690579] / Орга...,Радиоэлектронная промышленность [258],1037730020844,7.730170e+09,773001001,Москва [3505]


In [4]:
dfs.shape

(11387, 11)

In [162]:
tass_set = set()
maximum = 0
arr = []
for val in dfs['Вид деятельности, дополнительный ТАСС']:
    if type(val) == float:
        tass_set.add('-1')
        continue
    tass = val.split(' / ')
    if len(tass) > maximum:
        maximum = len(tass)
    arr.append(len(tass))
    for t in tass:
        tass_set.add(t.strip())
for val in dfs['Вид деятельности, основной ТАСС']:
    if type(val) == float:
        tass_set.add('-1')
        continue
    tass_set.add(val.strip())

In [160]:
okved_set = set()
maximum = 0
arr = []
for val in dfs['ОКВЭД2']:
    if type(val) == float:
        okved_set.add("-1")
        continue
    okved = val.split(' / ')
    if len(okved) > maximum:
        maximum = len(okved)
    arr.append(len(okved))
    for t in okved:
        okved_set.add(t.strip())

In [163]:
attr_set = set()
maximum = 0
arr = []
for val in dfs['Атрибуты предприятия']:
    if type(val) == float:
        attr_set.add('-1')
        continue
    attr = val.split(' / ')
    if len(attr) > maximum:
        maximum = len(attr)
    arr.append(len(attr))
    for t in attr:
        attr_set.add(t.strip())

In [164]:
otrasl_set = set()
maximum = 0
arr = []
for val in dfs['Отрасль']:
    if type(val) == float:
        otrasl_set.add('-1')
        continue
    otrasl = val.split(' / ')
    if len(otrasl) > maximum:
        maximum = len(otrasl)
    arr.append(len(otrasl))
    for t in otrasl:
        otrasl_set.add(t.strip())

In [148]:
len(list(otrasl_set))

60

In [149]:
len(list(attr_set))

9

In [150]:
len(list(okved_set))

909

In [151]:
len(list(tass_set))

2141

In [14]:
inns = set(dfs['ИНН'])

In [15]:
def load_json(path):
    DUMPS = []
    with open(path, 'r', encoding="utf8") as f:
        for data in f:
            dump = json.loads(data)
            inn_correct = False
            inn_correct = (len(dump['receiver'][0]['inn']) > 0 and np.int64(dump['receiver'][0]['inn']) in inns)
            if (len(dump['plans']) + len(dump['plansSubject']) + len(dump['addagreement'])) != 0 and inn_correct: 
                DUMPS.append(dump)
    return DUMPS

In [19]:
D2 = []
for path in os.listdir(r'data\dumps'):
    p = os.path.join(r'data\dumps', path)
    D2 += load_json(p)

In [20]:
len(D2)

14198

In [21]:
# t4 = []
# for dump in D2:
#     if dump['grbs']['fullName'] == 'МИНИСТЕРСТВО ПРОМЫШЛЕННОСТИ И ТОРГОВЛИ РОССИЙСКОЙ ФЕДЕРАЦИИ':
#         t4.append(dump)
# len(t4)

7151

In [97]:
columns = ['KBK'] + list(dfs.columns)
new_df = pd.DataFrame(columns=columns)
for dump in D2:
    if len(dump['plans'])  > 0:
        kbk = dump['plans'][0]['kbk']
    elif len(dump['plansSubject'])  > 0:
        kbk = dump['plansSubject'][0]['kbk']
    else:
        kbk = dump['addagreement'][0]['plantranssub'][0]['kbk']
        
    inn = np.int64(dump['receiver'][0]['inn'])
    data = dfs.loc[dfs['ИНН']==inn]
    data = [kbk] + list(data.iloc[0])
    
    df2 = pd.Series(data, index=columns)

    new_df = new_df.append(df2, ignore_index=True)

In [125]:
print(new_df.shape)
new_df.head()

(14198, 12)


,KBK,ОКВЭД2,"Вид деятельности, основной ТАСС","Вид деятельности, дополнительный ТАСС",Организационно правовая форма,Тип компании,Атрибуты предприятия,Отрасль,ОГРН,ИНН,КПП,Регион
0,02004111950168580812,NaN,Торговля оптовая производственным электротехни...,"Производство прочих химических продуктов, не в...",НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],NaN,Радиоэлектронная промышленность [258],1127847272629,7.814537e+09,NaN,Санкт-Петербург [3522]
1,02004121840168844452,Производство оружия и боеприпасов [27574],"Строительство кораблей, судов и плавучих конст...",Производство оружия и боеприпасов [27575] / Пр...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Организация ОПК [8780665] / Системообразующее ...,Судостроительная промышленность [262],1079847085966,7.838395e+09,783801001,Санкт-Петербург [3522]
2,73004122140062870451,Научные исследования и разработки в области ес...,Научные исследования и разработки в области ес...,Производство автоматических космических аппара...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Системообразующее предприятие [8690579] / Орга...,Транспортное машиностроение [264],1082452000290,2.452035e+09,245201001,Красноярский край [3496]
3,02006051610166750811,Производство автотранспортных средств [27850] ...,Производство автотранспортных средств [27850],"Производство двигателей и турбин, кроме авиаци...",ОБЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ [7026...,Организация [7026691],NaN,Автомобильная промышленность [246],1023900987626,3.906093e+09,NaN,Калининградская область [3485]
4,02004121616713810242,Производство автотранспортных средств [27850],Производство автотранспортных средств [27850],Торговля оптовая легковыми автомобилями и легк...,НЕПУБЛИЧНЫЕ АКЦИОНЕРНЫЕ ОБЩЕСТВА [7026704],Организация [7026691],Системообразующее предприятие [8690579],Автомобильная промышленность [246],1027739178202,7.709260e+09,772301001,Москва [3505]


In [139]:
new_df = new_df.drop(columns=["Тип компании"])

In [140]:
new_df.to_csv(r'data/data.csv', index=False)

In [120]:
kbk = list(new_df['KBK'].unique())
kbk_to_index = {key: value for key, value in enumerate(kbk)}
with open(r'kbk.json', 'w') as f:
    json.dump(kbk_to_index, f)

In [165]:
acting_to_index = {key+1: value for key, value in enumerate(tass_set)}
with open(r'acting.json', 'w') as f:
    json.dump(acting_to_index, f)

In [161]:
ocved_to_index = {key+1: value for key, value in enumerate(okved_set)}
with open(r'ocved.json', 'w') as f:
    json.dump(ocved_to_index, f)

In [166]:
attr_to_index = {key+1: value for key, value in enumerate(attr_set)}
with open(r'attr.json', 'w') as f:
    json.dump(attr_to_index, f)

In [167]:
otrasl_to_index = {key+1: value for key, value in enumerate(otrasl_set)}
with open(r'otrasl.json', 'w') as f:
    json.dump(otrasl_to_index, f)

In [168]:
forma_set = list(new_df['Организационно правовая форма'].unique())
forma_to_index = {key: value if type(value) != float else '-1' for key, value in enumerate(forma_set)}
with open(r'forma.json', 'w') as f:
    json.dump(forma_to_index, f)

In [169]:
region_set = list(new_df['Регион'].unique())
region_to_index = {key: value if type(value) != float else '-1' for key, value in enumerate(region_set)}
with open(r'region.json', 'w') as f:
    json.dump(region_to_index, f)